This notebook is for getting predicion residuals from models

In [69]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels
from scipy import stats
import scipy
import time
import matplotlib.pyplot as plt
import pickle
import random
import csv

from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, TensorDataset
# Import nn.functional
import torch.nn.functional as F

import sys

device = 'cpu'


In [ ]:
ind = sys.argv[1]
start = time.time()


In [2]:
# trim the gene labels ENSGXXXX.1  -> ENSGXXXX
def trimLabelss(names):
    return [i.split(".")[0] for i in names]

In [3]:
#import GTEx gene count matrix
normedMat = pd.read_csv("/gpfs/home/ydong/data/logTrimmedNormedCounts.csv",index_col = "Name")
normedMat = normedMat.transpose()

#trim gene labels ENSGXXXX.1  -> ENSGXXXX
normedMat.columns = trimLabelss(normedMat.columns)

In [5]:
genes = list(normedMat.columns)
currGeneL = list(pd.read_csv("/gpfs/home/ydong/data/GTEx/GTExProteinGenes/"+ind+".txt",header = None)[0])


In [59]:
#loop through the list and get residuals of prediction
outDF = pd.DataFrame(columns = currGeneL, index = list(normedMat.index))
for currGene in currGeneL:
    currModel = torch.load("/gpfs/home/ydong/data/mini/models/" +str(ind)+"/"+currGene+".sav")
    
    # Get residual
    currX = torch.from_numpy(normedMat.drop(currGene,axis = 1).to_numpy()).float().to(device)
    pred = currModel(currX).detach().numpy()
    Z = np.transpose(pred)[0]- list(normedMat[currGene])
    outDF[currGene] = Z


54.677330017089844

In [60]:
# perform a z test on models of genes
# Z score -> 2 sided P-value
absPs = []
Zs = []

for i in outDF.columns:
    geneZ = stats.zscore(outDF[i])
    Zs.append(geneZ)
    absPs.append(stats.norm.sf(abs(geneZ))*2)
absPs = np.array(absPs)

In [61]:
PDF = pd.DataFrame(absPs,index =currGeneL ,columns =list(normedMat.index)).transpose()
PDF.index.name = "GeneID"
PDF.to_csv("/gpfs/home/ydong/data/mini/absP/" +str(ind)+".csv")

In [68]:
end = time.time()
print(end - start)

227.63564610481262
